In [34]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn import linear_model,  svm, datasets


In [7]:
covid_df = pd.read_csv('covid.csv', index_col='id')
covid_df.head()

,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,diabetes,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,,,
16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,2,...,2,2,2,2,2,2,2,2,1,97
1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,2,...,2,2,2,2,2,2,2,99,1,97
167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,2,...,2,2,2,2,1,2,2,99,1,2
0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,2,...,2,2,2,2,2,2,2,99,1,2
0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,1,...,2,1,2,1,2,2,2,99,1,2


In [8]:
len(covid_df)

566602

In [10]:
# Drop any rows that are nan see what we're left with


In [11]:
covid_df.columns

Index(['sex', 'patient_type', 'entry_date', 'date_symptoms', 'date_died',
       'intubed', 'pneumonia', 'age', 'pregnancy', 'diabetes', 'copd',
       'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular',
       'obesity', 'renal_chronic', 'tobacco', 'contact_other_covid',
       'covid_res', 'icu'],
      dtype='object')

In [13]:
covid_df_small=covid_df.copy()
columns_to_drop_above_97=[
       'intubed', 'pneumonia', 'pregnancy', 'diabetes', 'copd',
       'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular',
       'obesity', 'renal_chronic', 'tobacco', 'contact_other_covid',
       'covid_res', 'icu']
# columns_to_drop_above_97=[
#        'intubed', 'pneumonia', '']
for col in columns_to_drop_above_97:
    covid_df_small = covid_df_small[covid_df_small[col]<97]





covid_df_small

,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,diabetes,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,,,
154fea,1,2,02-06-2020,02-06-2020,9999-99-99,2,2,25,2,2,...,2,2,2,2,2,2,2,1,1,2
16e14e,1,2,22-06-2020,17-06-2020,9999-99-99,2,2,52,2,2,...,2,2,2,2,1,2,1,1,1,2
149f1c,1,2,17-06-2020,12-06-2020,9999-99-99,2,1,51,2,2,...,2,2,2,2,2,2,2,1,1,2
1d5df2,1,2,08-06-2020,07-06-2020,9999-99-99,1,1,67,2,1,...,2,1,2,2,1,2,2,1,1,2
18f860,1,2,27-05-2020,27-05-2020,9999-99-99,2,1,59,2,1,...,2,2,2,2,2,2,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70426,1,2,19-06-2020,12-06-2020,9999-99-99,2,1,62,2,2,...,2,2,2,1,2,2,2,1,3,2
0143db,1,2,24-06-2020,21-06-2020,9999-99-99,2,2,35,2,2,...,2,2,2,2,1,2,2,1,3,2
188cd4,1,2,06-06-2020,01-06-2020,9999-99-99,2,1,57,2,2,...,2,1,2,2,1,2,2,2,3,2


In [ ]:
# did they die tho 
covid_df['Died?'] = covid_df['date_died'].apply(lambda x: 1 if x == '9999-99-99' else 2)

In [14]:

known_intubed = covid_df_small[covid_df_small.intubed < 97]

In [1]:
# What percentage of intubed is true
for index, row in covid_df_small.iterrows():
#     print(row.intubed)
    

SyntaxError: unexpected EOF while parsing (<ipython-input-1-f2afe23b5778>, line 4)

In [16]:
covid_df_small=covid_df.copy()
covid_df_small["intubed"]=covid_df_small["intubed"].apply(lambda x:"" if x>2 else x)
len(covid_df_small.dropna())

566602

In [ ]:
covid_df[0:10]

In [ ]:
from sklearn import svm

In [23]:
# Make array from 
columns_to_train=[ 'pneumonia', 'pregnancy', 'diabetes', 'copd',
       'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular',
       'obesity']
column_class=["intubed"]

In [28]:
x=covid_df[columns_to_train].to_numpy()
y=covid_df["intubed"].to_numpy()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=45)


In [31]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train) 


# predict the response
knn_prediction = knn_classifier.predict(X_test)

In [33]:
print(accuracy_score(y_test, knn_prediction))

0.8610740481888586


In [35]:
# Logistic regression

# Create logistic regression object
# Try different solvers:  'newton-cg', 'sag', 'saga' and 'lbfgs'
logistic_regression_classifier = linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')

# Train the model using the training sets
logistic_regression_classifier.fit(X_train,y_train)

# Make predictions using the testing set
logistic_regression_classifier_prediction = logistic_regression_classifier.predict(X_test)


# evaluate accuracy
print(accuracy_score(y_test, logistic_regression_classifier_prediction))


0.8733436403555217


/Users/claireh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
